## SAS Function to Python using GenAI

### Problem Statement

In [2]:
%pip install scikit-learn
%pip install openai
# Install required modules and set required tokens
%pip install openai python-dotenv
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Load the .env file in a Jupyter notebook:
%load_ext dotenv
%dotenv

In [5]:
import pandas as pd
import openai
import os
import json

### Setup proxy

In [19]:
import requests
from requests.auth import HTTPProxyAuth

ModuleNotFoundError: No module named 'requests'

In [20]:
# Configure your proxy
proxy = "http://95.164.128.235:9943"
proxy_auth = HTTPProxyAuth('M0k0EN', 'ZSNkKy')

# Configure a session to use the proxy
session = requests.Session()
session.proxies = {'http': proxy, 'https': proxy}
session.auth = proxy_auth

NameError: name 'HTTPProxyAuth' is not defined

In [6]:
openai.api_key = os.environ.get('OPENAI_API_KEY')
openai.organization = os.environ.get('OPENAI_ORG_ID')
if os.environ.get('use_proxy'):
    openai.session = session


In [16]:
# https://sebscholl.medium.com/using-chatgpt-in-jupyter-notebooks-c6144fef79d8
class ChatSession:
    """
    Class for running a chat session. 
    
    Create and use a new chat by running:
        chat = ChatSession('new-chat-name', 'system-role-message')

    Load an existing chat by running:
        chat = ChatSession('existing-chat-name')
    """
    model = 'gpt-3.5-turbo-16k-0613'
    
    def __init__(self, chat_name, system_role=None):
        self.name = chat_name
        self.path = f"./chats/{chat_name}.json"

        if system_role is not None:
            with open(self.path, "w") as outfile:
                outfile.write(json.dumps([system_role], indent=4))
            print(f"Chat {self.path} created 💬")
        else:
            print(f"Active chat set to {self.name} 💬")
            
    def chat(self, message):
        # Retrieve chat history
        with open(self.path) as openchat:
            chat_history = json.load(openchat)
        # Update chat history before completion
        chat_history.append({'role': 'user', 'content': message})
        # Run completeion
        chat_completion = openai.completions.create(model=self.model, prompt=chat_history)
        # Save message to chat
        chat_history.append(chat_completion.choices[0].message)
        # Handle lenth finish reason
        if chat_completion.choices[0].finish_reason == 'length':
            chat_completion = openai.completions.create(model=self.model, prompt=chat_history)
        # Save the chat history
        with open(self.path, "w") as outfile:
            json.dump(chat_history, outfile)
        # Print new dialog.
        print(chat_history[-2:])

### Start chat session

In [17]:
c1 = ChatSession("philosophical-chat", { "role": "system", "content": "You are an AI assistant that converts SAS code to Python." }) 

Chat ./chats/philosophical-chat.json created 💬


In [18]:
c1.chat("""
/* Assign some macro variables */

%let var1 = test;

%let id = 39;

/* Print to log */

%put var1 = &var1.;

%put id = &id.;

/* Use the macro variables in a data step to create a 1 record table */

data test;

       m&id. = 333;

       v = "&var1.";

run;
""")

APIConnectionError: Connection error.